In [2]:
!pip install selenium
!pip install webdriver-manager


   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.7 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.7 MB 1.7 MB/s eta 0:00:06
   ---- ----------------------------------- 1.0/9.7 MB 1.7 MB/s eta 0:00:05
   ------ --------------------------------- 1.6/9.7 MB 1.7 MB/s eta 0:00:05
   ------- -------------------------------- 1.8/9.7 MB 1.7 MB/s eta 0:00:05
   --------- ------------------------------ 2.4/9.7 MB 1.8 MB/s eta 0:00:05
   ---------- ----------------------------- 2.6/9.7 MB 1.8 MB/s eta 0:00:04
   ------------ --------------------------- 3.1/9.7 MB 1.8 MB/s eta 0:00:04
   --------------- ------------------------ 3.7/9.7 MB 1.9 MB/s eta 0:00:04
   ----------------- ---------------------- 4.2/9.7 MB 1.9 MB/s eta 0:00:03
   ------------------ --------------------- 4.5/9.7 MB 1.9 MB/s eta 0:00:03
   -------------------- ---------

In [2]:
# scrape_caqueta_full.py
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time, re

HEADLESS = True
OUT_CSV = "caqueta_full.csv"
BASE_PAGE_URL = "https://caqueta.travel/single-category/page/{}/"

def setup_driver(headless=True):
    opts = webdriver.ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--window-size=1920,1080")
    opts.add_argument("user-agent=Mozilla/5.0")
    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

def safe_text(el):
    try:
        return el.text.strip()
    except:
        return ""

def get_cards(driver):
    cards = driver.find_elements(By.CSS_SELECTOR, "article.directorist-listing-single")
    results = []
    for art in cards:
        try:
            a = art.find_element(By.CSS_SELECTOR, "h2.directorist-listing-title a")
            title = safe_text(a)
            link = a.get_attribute("href")
        except:
            continue
        try:
            city_el = art.find_element(By.CSS_SELECTOR, "li.directorist-listing-card-location a")
            city = safe_text(city_el)
        except:
            city = ""
        results.append({"title": title, "link": link, "city": city})
    return results

def parse_detail(driver, url):
    main_handle = driver.current_window_handle
    driver.execute_script("window.open('');")
    new_handle = [h for h in driver.window_handles if h != main_handle][-1]
    driver.switch_to.window(new_handle)
    detail = {}

    try:
        driver.get(url)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h1, .directorist-listing-details__listing-title"))
        )

        # título detalle
        try:
            detail["detail_title"] = safe_text(driver.find_element(By.CSS_SELECTOR, "h1.directorist-listing-details__listing-title"))
        except:
            detail["detail_title"] = driver.title

        # categorías
        cats = []
        for c in driver.find_elements(By.CSS_SELECTOR, ".directorist-listing-category a"):
            txt = safe_text(c)
            if txt:
                cats.append(txt)
        detail["categories"] = ", ".join(cats)

        # dirección
        try:
            detail["address"] = safe_text(driver.find_element(By.CSS_SELECTOR, ".field-name-field-lugar-o-direccion"))
        except:
            detail["address"] = ""

        # teléfonos
        phones = []
        for sel in [".directorist-single-info-phone a", ".directorist-single-info-phone2 a", ".field-name-field-telefonos"]:
            try:
                el = driver.find_element(By.CSS_SELECTOR, sel)
                val = safe_text(el)
                if val:
                    phones.append(val)
            except:
                pass
        detail["phones"] = " | ".join(phones)

        # email
        try:
            detail["email"] = safe_text(driver.find_element(By.CSS_SELECTOR, ".directorist-single-info-email a"))
        except:
            detail["email"] = ""

        # descripción
        try:
            detail["description"] = safe_text(driver.find_element(By.CSS_SELECTOR, ".directorist-details-info-wrap"))
        except:
            detail["description"] = ""

        # coordenadas
        detail["lat"], detail["lon"] = None, None
        for a in driver.find_elements(By.CSS_SELECTOR, "a[href*='daddr=']"):
            href = a.get_attribute("href") or ""
            m = re.search(r"daddr=([-\d\.]+)\s*,\s*([-\d\.]+)", href)
            if m:
                detail["lat"], detail["lon"] = m.group(1), m.group(2)
                break

        # imágenes
        imgs = []
        for im in driver.find_elements(By.CSS_SELECTOR, ".directorist-single-listing-slider img, .directorist-thumnail-card-front-img"):
            src = im.get_attribute("src")
            if src:
                imgs.append(src)
        detail["images"] = " | ".join(imgs)

    except Exception as e:
        print("   ⚠️ Error en detalle:", e)

    finally:
        driver.close()
        driver.switch_to.window(main_handle)

    return detail

def scrape_all(start_page=1, max_pages=20):
    driver = setup_driver(HEADLESS)
    data, seen_titles = [], set()

    try:
        driver.get(BASE_PAGE_URL.format(start_page))
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "article.directorist-listing-single"))
        )

        page = start_page
        while page <= max_pages:
            print(f"[+] Página {page} -> {driver.current_url}")
            cards = get_cards(driver)
            if not cards:
                break

            new_cards = [c for c in cards if c["title"] not in seen_titles]
            if not new_cards:
                print("⚠️ Página repetida, fin.")
                break

            for c in new_cards:
                seen_titles.add(c["title"])
                detail = parse_detail(driver, c["link"])
                row = {**c, **detail}
                data.append(row)
                print("   >", c["title"], "|", c["city"], "|", detail.get("phones", ""))

                time.sleep(0.8)

            # ir a la siguiente página
            page += 1
            driver.get(BASE_PAGE_URL.format(page))

            # verificar que cambió el primer título
            try:
                WebDriverWait(driver, 10).until_not(
                    EC.text_to_be_present_in_element(
                        (By.CSS_SELECTOR, "h2.directorist-listing-title a"),
                        new_cards[0]["title"]
                    )
                )
            except:
                print("✅ Fin: no cambian los títulos.")
                break

            time.sleep(2)

    finally:
        driver.quit()

    df = pd.DataFrame(data)
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"[✔] Guardados {len(df)} registros en {OUT_CSV}")
    return df

if __name__ == "__main__":
    scrape_all(start_page=1, max_pages=15)


[+] Página 1 -> https://caqueta.travel/single-category/
   > Hotel Bacata Real | Florencia | (098) /4356889 | (57) 3125055843 | Teléfonos: (098) /4356889
   > La Calera Amazónica | Florencia | (8) 4370924 | 3132051224 - 3102889948 | Teléfonos: (8) 4370924
   > Ferry MarcoPolo Crucero Fluvial Ecológico | Florencia | 3157821419 | 3164110752
   > Parque Ecológico El Paraíso de Pedro | Florencia | 3203000436
   > Reserva Natural Dos Quebradas | Florencia | 3147420111
   > La Calera Amazónica – Morelia Caquetá | Morelia | 3108768495 | 3134416609
[+] Página 2 -> https://caqueta.travel/single-category/page/2/
   > Hotel Royal Plaza | Florencia | (098) 435 5848 - 435 7504 | (57)3174418114 - 3174419131 | Teléfonos: (098) 435 5848 - 435 7504
   > Parroquia Nuestra Señora de la Consolata | Albania | 4303101 | Teléfonos: 4303101
   > Santuario Virgen de las Mercedes | Albania | 
   > Palacio Municipal | Albania | 430 3089 | Teléfonos: 430 3089
   > Centro Regional de Educación Superior de Albania 

In [3]:
import pandas as pd

# Leer el CSV generado
df = pd.read_csv("caqueta_full.csv")

# Eliminar columnas lat y lon
df_clean = df.drop(columns=["lat", "lon"], errors="ignore")

# Guardar nuevo archivo
df_clean.to_csv("caqueta_full_sin_coordenadas.csv", index=False, encoding="utf-8-sig")

print("Archivo guardado como caqueta_full_sin_coordenadas.csv")
df_clean.head()


Archivo guardado como caqueta_full_sin_coordenadas.csv


,title,link,city,detail_title,categories,address,phones,email,description,images
0,Hotel Bacata Real,https://caqueta.travel/directorio/hotel-bacata...,Florencia,HOTEL BACATA REAL,"Agencias, Hoteles, Hoteles",Lugar o Dirección: Calle16 No. 12 – 49 Centro ...,(098) /4356889 | (57) 3125055843 | Teléfonos: ...,hotelroyalplazaflorencia@hotmail.com,Lugar o Dirección: Calle16 No. 12 – 49 Centro ...,https://caqueta.travel/wp-content/uploads/2024...
1,La Calera Amazónica,https://caqueta.travel/directorio/la-calera-am...,Florencia,LA CALERA AMAZÓNICA,"Actractivos Turisticos, Bar, Actractivos Turis...",Lugar o Dirección: Kilómetro 4 Vía Sebastopol ...,(8) 4370924 | 3132051224 - 3102889948 | Teléfo...,lacaleraamazonica@hotmail.com,Lugar o Dirección: Kilómetro 4 Vía Sebastopol ...,https://caqueta.travel/wp-content/uploads/2024...
2,Ferry MarcoPolo Crucero Fluvial Ecológico,https://caqueta.travel/directorio/ferry-marcop...,Florencia,FERRY MARCOPOLO CRUCERO FLUVIAL ECOLÓGICO,"Actractivos Turisticos, Actractivos Turisticos...","Lugar o Dirección: Vda Puerto Arango, Corregim...",3157821419 | 3164110752,ferrymarcopolo27@gmail.com,"Lugar o Dirección: Vda Puerto Arango, Corregim...",https://caqueta.travel/wp-content/uploads/2024...
3,Parque Ecológico El Paraíso de Pedro,https://caqueta.travel/directorio/parque-ecolo...,Florencia,PARQUE ECOLÓGICO EL PARAÍSO DE PEDRO,"Finca, Actractivos Turisticos, Actractivos Tur...",Lugar o Dirección: Vereda El Limón,3203000436,paraisodepedro@hotmail.com,Lugar o Dirección: Vereda El Limón\nPersona de...,https://caqueta.travel/wp-content/uploads/2024...
4,Reserva Natural Dos Quebradas,https://caqueta.travel/directorio/reserva-natu...,Florencia,RESERVA NATURAL DOS QUEBRADAS,"Finca, Actractivos Turisticos, Actractivos Tur...",Lugar o Dirección: Finca Dos Quebradas – Vered...,3147420111,NaN,Lugar o Dirección: Finca Dos Quebradas – Vered...,https://caqueta.travel/wp-content/uploads/2024...


In [20]:
import pandas as pd

# Rutas
path_simple = "../acotur_huila_competitiva/output/CAQUETA.csv"
path_full = "caqueta_full_sin_coordenadas.csv"

# Leer ambos
df_simple = pd.read_csv(path_simple)
df_full = pd.read_csv(path_full)

# Agregar columna Fuente
df_simple["Fuente"] = "Acotur"
df_full["Fuente"] = "Caqueta Travel"

# Renombrar para alinear (opcional, así queda más consistente)
df_full_ren = df_full.rename(columns={
    "title": "Nombre",
    "link": "URL",
    "city": "Ciudad",
    "detail_title": "Detalle",
    "categories": "Categorías",
    "address": "Dirección",
    "phones": "Teléfonos",
    "email": "Email",
    "description": "Descripción",
    "images": "Imágenes"
})

# Concatenar filas
df_concat = pd.concat([df_simple, df_full_ren], ignore_index=True)

# Reordenar columnas: Fuente primero
cols = ["Fuente"] + [c for c in df_concat.columns if c != "Fuente"]
df_concat = df_concat[cols]

# Guardar archivo final
df_concat.to_csv("CAQUETA_UNION.csv", index=False, encoding="utf-8-sig")

print("✅ Archivo unido guardado como CAQUETA_UNION.csv")
print("Columnas finales:", df_concat.columns.tolist())


✅ Archivo unido guardado como CAQUETA_UNION.csv
Columnas finales: ['Fuente', 'Nombre', 'Municipio', 'RNT', 'Descripción', 'Categorías', 'Certificaciones', 'Email', 'Redes Sociales', 'URL', 'Departamento', 'Ciudad', 'Detalle', 'Dirección', 'Teléfonos', 'Imágenes']


In [21]:
excel_file = "../acotur_huila_competitiva/output/excel/portuColombia_final.xlsx"   # tu archivo existente con varias hojas
# Escribir como nueva hoja en el Excel existente
with pd.ExcelWriter(excel_file, mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
    df_concat.to_excel(writer, sheet_name="Caquetá", index=False)

print("✅ Hoja 'CAQUETA_UNION' fue sobrescrita en el archivo", excel_file)

✅ Hoja 'CAQUETA_UNION' fue sobrescrita en el archivo ../acotur_huila_competitiva/output/excel/portuColombia_final.xlsx
